In [1]:
from bs4 import BeautifulSoup
import json
import os
import requests
import subprocess
import sys

In [2]:
# for each repo, make gh api request to get ready-made info 
"""
https://docs.github.com/en/rest/repos/repos?apiVersion=2022-11-28#get-a-repository
"""
def make_gh_api_repo_req(owner_repo):
    header_accept = "Accept: application/vnd.github+json"
    header_gh_version = "X-GitHub-Api-Version: 2022-11-28"
    req_owner_repo = "/repos/" + owner_repo
    process = subprocess.Popen(['gh', 'api',
                                '-H', header_accept,
                                '-H', header_gh_version,
                                req_owner_repo],
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
    print("[LOG] Running", process.args)
    stdout, stderr = process.communicate()
    response = json.loads(stdout.decode("utf-8"))
    print(stderr.decode("utf-8"))
    repo = extract_details_from_api_response(response)
    repo["latest_version"] = make_gh_api_latest_release_req(owner_repo)
    return repo

In [5]:
# for each repo, make gh api request to get latest release version, if available, or th latest commit
"""
https://docs.github.com/en/rest/releases/releases?apiVersion=2022-11-28#get-the-latest-release
"""
def make_gh_api_latest_release_req(owner_repo):
    release = {}
    header_accept = "Accept: application/vnd.github+json"
    header_gh_version = "X-GitHub-Api-Version: 2022-11-28"
    req_owner_repo_latest_release = "/repos/" + owner_repo + "/releases/latest"
    process = subprocess.Popen(['gh', 'api',
                                '-H', header_accept,
                                '-H', header_gh_version,
                                req_owner_repo_latest_release],
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
    print("[LOG] Getting latest release with", process.args)
    stdout, stderr = process.communicate()
    response = json.loads(stdout.decode("utf-8"))
    print(stderr.decode("utf-8"))
    
    if ("404" not in stderr.decode("utf-8")): 
        # a release exists
        return response["tag_name"]
    else:
        # tags may not be sorted chronologically, so we get the latest commit
        req_owner_repo_commits = "/repos/" + owner_repo + "/commits"
        process = subprocess.Popen(['gh', 'api',
                                    '-H', header_accept,
                                    '-H', header_gh_version,
                                    req_owner_repo_commits],
                                   stdout=subprocess.PIPE, 
                                   stderr=subprocess.PIPE)
        print("[LOG] Getting list of commits with", process.args)
        stdout, stderr = process.communicate()
        response = json.loads(stdout.decode("utf-8"))
        print(stderr.decode("utf-8"))
        return response[0]["sha"]

In [6]:
def extract_details_from_api_response(response):
    repo = {}
    repo["full_name"] = response["full_name"]
    repo["name"] = response["name"]
    repo["html_url"] = response["html_url"]
    repo["ssh_url"] = response["ssh_url"]
    repo["stargazers_count"] = response["stargazers_count"]
    repo["forks_count"] = response["forks_count"]
    repo["topics"] = response["topics"]
    repo["language"] = response["language"]
    repo["decription"] = response["description"]
    repo = extract_details_from_repo_html(repo)
    return repo

In [7]:
def get_span_for_commits(span):
    return span.has_attr('aria-label') and "Commits on " in span.get('aria-label')

def get_span_for_latest_release(span):
    return span.has_attr('title') and span.get('title') == "Label: Latest"

# Parse html_url to get the number of contributors and commits
def extract_details_from_repo_html(repo):
    repo_gh_page = requests.get(repo["html_url"])
    soup = BeautifulSoup(repo_gh_page.text, 'html.parser')
    
    # contributors
    """
    <a href="/<full_name>/graphs/contributors>
      <span title="<num_contributors" ...><num_contributors></span>
    </a>
    """
    for link in soup.find_all('a'):
        if link.get('href') == "/" + repo["full_name"] + "/graphs/contributors":
            repo["contributors"] = int(link.span.string.replace(",", ""))
            break
    
    
    # commits
    """
    <span ...>
      <strong><num_commits></strong>
      <span aria-label="Commits on <default_branch>"...>commits</span>
    </span>
    """
    for a_span in soup.find_all(get_span_for_commits):
        repo["commits"] = int(a_span.parent.strong.string.replace(",", ""))
        break
    
    return repo

# Repositories that list `#testing` or `#test` as a topic

- testing-related (manually judged)
- non-deprecrated/archived
- primary language of README is English
- \>= 1K stars

In [21]:
"""
Java

https://github.com/topics/testing?l=java
https://github.com/topics/test?l=java

"""

seed_java = [
    "mockito/mockito",
    "testcontainers/testcontainers-java",
    "karatelabs/karate",
    "powermock/powermock",
    "awaitility/awaitility",
    "spockframework/spock",
    "assertj/assertj",
    "Wisser/Jailer",
    "testng-team/testng",
    "hcoles/pitest",
    "sqlancer/sqlancer",
    "fabioCollini/DaggerMock",
    "linkedin/test-butler",
    "apache/jmeter",
    "rest-assured/rest-assured"
]

print("Java projects:", len(seed_java))

Java projects: 15


In [22]:
"""
JavaScript

https://github.com/topics/testing?l=javascript
https://github.com/topics/test?l=javascript

"""

seed_javascript = [
    "cypress-io/cypress",
    "mochajs/mocha",
    "avajs/ava",
    "enzymejs/enzyme",
    "testing-library/react-testing-library",
    "jasmine/jasmine",
    "nightwatchjs/nightwatch",
    "wix/Detox",
    "Netflix/pollyjs",
    "artilleryio/artillery",
    "miragejs/miragejs",
    "apiaryio/dredd",
    "testing-library/jest-dom",
    "qunitjs/qunit",
    "codeceptjs/CodeceptJS",
    "pa11y/pa11y",
    "americanexpress/jest-image-snapshot",
    "testing-library/dom-testing-library",
    "jhnns/rewire",
    "pod4g/hiper",
    "ChangemakerStudios/Papercut-SMTP",
    "jest-community/jest-extended",
    "DeviceFarmer/stf",
    "bbyars/mountebank",
    "fent/randexp.js",
    "testing-library/cypress-testing-library",
    "oblador/loki",
    "andywer/leakage",
    "vlucas/frisby",
    "pixielabs/cavy",
    "jfairbank/redux-saga-test-plan",
    "Pythagora-io/pythagora",
    "developit/karmatic",
    "atlassian/jest-in-case",
    "DevExpress/testcafe",
    "marmelab/gremlins.js",
    "angular/protractor",
    "bahmutov/start-server-and-test"    
]

print("JavaScript projects:", len(seed_javascript))

JavaScript projects: 38


In [23]:
"""
TypeScript

https://github.com/topics/testing?l=typescript
https://github.com/topics/test?l=typescript

"""

seed_typescript = [
    "puppeteer/puppeteer",
    "storybookjs/storybook",
    "microsoft/playwright",
    "hoppscotch/hoppscotch",
    "jestjs/jest",
    "react-cosmos/react-cosmos",
    "kulshekhar/ts-jest",
    "testing-library/react-hooks-testing-library",
    "theintern/intern",
    "dubzzz/fast-check",
    "qawolf/qawolf",
    "garden-io/garden",
    "callstack/react-native-testing-library",
    "stryker-mutator/stryker-js",
    "nodkz/mongodb-memory-server",
    "nvh95/jest-preview",
    "tajo/ladle",
    "benkeen/generatedata",
    "testing-library/user-event",
    "MockyAbstract/Mocky",
    "ngneat/spectator",
    "nut-tree/nut.js",
    "open-source-labs/spearmint",
    "testcontainers/testcontainers-node",
    "stepci/stepci",
    "vitest-dev/vitest",
    "webdriverio/webdriverio",
    "Thiht/smocker"
]

print("TypeScript projects:", len(seed_typescript))

TypeScript projects: 28


In [24]:
"""
Python

https://github.com/topics/testing?l=python
https://github.com/topics/test?l=python

"""

seed_python = [
    "localstack/localstack",
    "joke2k/faker",
    "pytest-dev/pytest",
    "robotframework/robotframework",
    "HypothesisWorks/hypothesis",
    "ultrafunkamsterdam/undetected-chromedriver",
    "pywinauto/pywinauto",
    "lk-geimfari/mimesis",
    "ansible/molecule",
    "tox-dev/tox",
    "FactoryBoy/factory_boy",
    "unionai-oss/pandera",
    "kevin1024/vcrpy",
    "pytest-dev/pytest-testinfra",
    "gabrielfalcao/HTTPretty",
    "powerfulseal/powerfulseal",
    "schemathesis/schemathesis",
    "wkeeling/selenium-wire",
    "dtmilano/AndroidViewClient",
    "scanapi/scanapi",
    "terraform-compliance/cli",
    "darrenburns/ward",
    "Teemu/pytest-sugar",
    "wntrblm/nox"
]

print("Python projects:", len(seed_python))

Python projects: 24


In [25]:
"""
C

https://github.com/topics/testing?l=c
https://github.com/topics/test?l=c

"""

seed_c = [
    "session-replay-tools/tcpcopy",
    "AFLplusplus/AFLplusplus"
]

print("C projects:", len(seed_c))

C projects: 2


In [26]:
"""
C++

https://github.com/topics/testing?l=cpp
https://github.com/topics/test?l=cpp

"""

seed_cpp = [
    "catchorg/Catch2",
    "doctest/doctest",
    "0vercl0k/wtf",
    "microsoft/pict",
    "boost-ext/ut"
]

print("C++ projects:", len(seed_cpp))

C++ projects: 5


In [27]:
"""
C#

https://github.com/topics/testing?l=c%23
https://github.com/topics/test?l=c%23

"""

seed_csharp = [
    "testcontainers/testcontainers-dotnet",
    "nunit/nunit",
    "nsubstitute/NSubstitute",
    "danielpalme/ReportGenerator",
    "VerifyTests/Verify",
    "shouldly/shouldly",
    "FlaUI/FlaUI",
    "richardszalay/mockhttp",
    "microsoft/coyote",
    "TestableIO/System.IO.Abstractions",
    "WireMock-Net/WireMock.Net",
    "tonerdo/pose",
    "mariotoffia/FluentDocker"
]

print("C# projects:", len(seed_csharp))

C# projects: 13


In [28]:
"""
Rust

https://github.com/topics/testing?l=rust
https://github.com/topics/test?l=rust

"""

seed_rust = [
    "Orange-OpenSource/hurl",
    "nextest-rs/nextest"
]

print("Rust projects:", len(seed_rust))

Rust projects: 2


In [29]:
"""
Go

https://github.com/topics/testing?l=go
https://github.com/topics/test?l=go

"""

seed_go = [
    "stretchr/testify",
    "buger/goreplay",
    "chromedp/chromedp",
    "Shopify/toxiproxy",
    "onsi/ginkgo",
    "gruntwork-io/terratest",
    "DATA-DOG/go-sqlmock",
    "goss-org/goss",
    "vektra/mockery",
    "google/syzkaller",
    "cweill/gotests",
    "dvyukov/go-fuzz",
    "go-rod/rod",
    "qustavo/httplab",
    "google/go-cmp",
    "open-policy-agent/conftest",
    "alexei-led/pumba",
    "testcontainers/testcontainers-go",
    "gavv/httpexpect",
    "SpectoLabs/hoverfly",
    "keploy/keploy",
    "google/martian",
    "h2non/gock",
    "matryer/is",
    "gotestyourself/gotestsum",
    "matryer/moq",
    "sanity-io/litter",
    "orlangure/gnomock",
    "helm/chart-testing",
    "qax-os/goreporter"
]

print("Go projects:", len(seed_go))

Go projects: 30


In [30]:
"""
PHP

https://github.com/topics/testing?l=php
https://github.com/topics/test?l=php

"""

seed_php = [
    "pestphp/pest",
    "Codeception/Codeception",
    "orchestral/testbench",
    "infection/infection",
    "laravel/dusk",
    "bovigo/vfsStream",
    "atoum/atoum",
    "kahlan/kahlan"
]

print("PHP projects:", len(seed_php))

PHP projects: 8


In [31]:
"""
Ruby

https://github.com/topics/testing?l=ruby
https://github.com/topics/test?l=ruby

"""

seed_ruby = [
    "thoughtbot/factory_bot",
    "thoughtbot/shoulda-matchers",
    "minitest/minitest",
    "DatabaseCleaner/database_cleaner",
    "inspec/inspec",
    "test-kitchen/test-kitchen",
    "test-prof/test-prof",
    "thoughtbot/appraisal",
    "freerange/mocha",
    "rubycdp/cuprite",
    "travisjeffery/timecop"
]

print("Ruby projects:", len(seed_ruby))

Ruby projects: 11


In [20]:
# prepare seed data with owner/repo per lang
repos = []

seed_repos = [
    seed_java,
    seed_javascript,
    seed_typescript,
    seed_python,
    seed_c,
    seed_cpp,
    seed_csharp,
    seed_go,
    seed_rust,
    seed_ruby,
    seed_php
]

# Make gh api request for each owner/repo
for i in range(len(seed_repos)):
    print("[LOG] Working on", seed_repos[i])
    for seed_repo in seed_repos[i]:
        repo = make_gh_api_repo_req(seed_repo)
        repos.append(repo)
        print("---------------------------------------------------")

with open("./repo_details.json", "w") as outfile:
    json.dump(repos, outfile, indent=2)

[LOG] Working on ['mockito/mockito', 'testcontainers/testcontainers-java', 'karatelabs/karate', 'powermock/powermock', 'awaitility/awaitility', 'spockframework/spock', 'assertj/assertj', 'Wisser/Jailer', 'testng-team/testng', 'hcoles/pitest', 'sqlancer/sqlancer', 'fabioCollini/DaggerMock', 'linkedin/test-butler', 'apache/jmeter', 'rest-assured/rest-assured']
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/mockito/mockito']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/mockito/mockito/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/testcontainers/testcontainers-java']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitH


[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/mochajs/mocha/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/avajs/ava']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/avajs/ava/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/enzymejs/enzyme']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/enzymejs/enzyme/releases/latest']
gh: Not Found (HTTP 404)

[LOG] Getting list of commits with ['gh', 'api', '-H', 'Accept: ap


[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/ChangemakerStudios/Papercut-SMTP/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/jest-community/jest-extended']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/jest-community/jest-extended/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/DeviceFarmer/stf']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/DeviceFarmer/stf/releases/latest']

--------------------------------


[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/puppeteer/puppeteer/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/storybookjs/storybook']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/storybookjs/storybook/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/microsoft/playwright']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/microsoft/playwright/releases/latest']

---------------------------------------------------


[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/nut-tree/nut.js/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/open-source-labs/spearmint']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/open-source-labs/spearmint/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/testcontainers/testcontainers-node']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/testcontainers/testcontainers-node/releases/latest']

-----------------


[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/kevin1024/vcrpy/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/pytest-dev/pytest-testinfra']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/pytest-dev/pytest-testinfra/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/gabrielfalcao/HTTPretty']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/gabrielfalcao/HTTPretty/releases/latest']
gh: Not Found (HTTP 404)

[LOG] Gettin


---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/nunit/nunit']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/nunit/nunit/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/nsubstitute/NSubstitute']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/nsubstitute/NSubstitute/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/danielpalme/ReportGenerator']

[LOG] Getting latest release with ['gh', 


[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/goss-org/goss/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/vektra/mockery']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/vektra/mockery/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/google/syzkaller']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/google/syzkaller/releases/latest']
gh: Not Found (HTTP 404)

[LOG] Getting list of commits with ['gh', 'api', '-H',


---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/helm/chart-testing']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/helm/chart-testing/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/qax-os/goreporter']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/qax-os/goreporter/releases/latest']

---------------------------------------------------
[LOG] Working on ['Orange-OpenSource/hurl', 'nextest-rs/nextest']
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/Oran


[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/orchestral/testbench/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/infection/infection']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/infection/infection/releases/latest']

---------------------------------------------------
[LOG] Running ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/laravel/dusk']

[LOG] Getting latest release with ['gh', 'api', '-H', 'Accept: application/vnd.github+json', '-H', 'X-GitHub-Api-Version: 2022-11-28', '/repos/laravel/dusk/releases/latest']

---------------------------------------------------
[LOG] Running ['gh

## TODO:
- clone repo
- get LOC with `cloc`
- run tests to get number of tests (?)
- language-wise code analysis to extract
  - test inputs (literals)
  - test variable names
  - assertion messages
  - comments